# **serve** with `tornado`

In [ ]:
%%file app.py
import asyncio
from tornado import web
import fib

class FibHandler(web.RequestHandler):
    def get(self, n):
        self.write({"result": fib.fib(int(n))})

async def main():
    app = web.Application([
        (r"/fib/(?P<n>\d*)", FibHandler),
        (r"/static/(.*)", web.StaticFileHandler, {"path": "."}),
    ])
    app.listen(8899)
    await asyncio.Event().wait()

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
import asyncio, anyio
async def serve():
    await anyio.run_process(["python", "app.py"])
asyncio.create_task(serve())

## **request** with a client

In [ ]:
from tornado.httpclient import AsyncHTTPClient
while True:
    try: r = await AsyncHTTPClient().fetch(
        "http://127.0.0.1:8899/fib/10"
    ); break
    except: await anyio.sleep(1)
r.body

# **acceptance** test with `robotframework`

In [ ]:
%reload_ext JupyterLibrary

In [ ]:
%%robot
*** Settings ***
Library           SeleniumLibrary

*** Tasks ***
Get the 10th Fibonacci Number
    Open Browser  http://127.0.0.1:8899/fib/10
    Wait Until Page Contains   {"result": 55}
    Capture Page Screenshot
    [Teardown]   Close All Browsers

# **conformance** test with `schemathesis`

In [ ]:
%%file openapi.yml
openapi: 3.0.0
info:
  title: fibonacci
  version: 0.0.0
paths:
  /fib/{n}:
    get:
      parameters:
        - name: n
          in: path
          required: true
          schema:
            type: string
            pattern: \d{1}
      responses:
        '200':
          content:
            application/json:
              schema:
                type: object
                require: [result]
                properties:
                  result:
                    type: integer

In [ ]:
!st run http://127.0.0.1:8899/static/openapi.yml #--hypothesis-verbosity verbose

# **load** test with `locust`

In [ ]:
%%file locustfile.py
import random
from locust import HttpUser, task, between

class Fibber(HttpUser):
    host = "http://127.0.0.1:8899"
    wait_time = between(1, 5)
    @task
    def hello_world(self):
        self.client.get(
            f"/fib/{random.randint(1, 10)}"
        )

In [ ]:
!locust -f locustfile.py --headless --users 10 --run-time 5 # seconds

# Next
[Managing entropy](./03-entropy.ipynb)